# CVAE Implementation

References:

[1] Sohn, Kihyuk, Honglak Lee, and Xinchen Yan. "Learning structured output representation using deep conditional generative models." Advances in neural information processing systems. 2015.

## Variational Auto-Encoders

The Conditional Variational Auto-Encoder is a latent variable model which assumes the existence of a prior latent variable $z$ from which the target data $y$ is generated. 

### Formulation 
The formulation is a variant of the VAE for which the generative process is assumed to be: 
* $z \sim p_\theta(z) $
* $y \sim p_\theta(y \vert z)$
where the probabilities are parameterized by a variable $\theta$, which often represent the parameters of a neural network. $p_\theta(z)$ is called the **prior** on $z$.  $p_\theta(y \vert z)$ is called the **marginal likelihood** of the data. 

Learning to generate samples from a given dataset $(x_i)_i \in \mathcal{D}$ by maximizing the likelihood $p_\theta(y)$ is generally intractable because it would require estimating the posterior distribution:
$$ p_\theta(y) = \frac{p_\theta(y\vert z)p_\theta(z)}{p_\theta(z\vert y)} $$.
This is intractable because it requires integrating over all possible values of the latent variable $z$. 

### Variational Lower Bound 

In practice, VAEs can be trained efficiently using a variational lower bound. Since the posterior distribution is intractable, it is approximated by a proposal distribution $q_\phi(z \vert y)$ which is assumed to be tractable to evaluate.

The log-likelihood can then be rewritten using the proposal distribution:

$$ \log p_\theta(y) = \mathbb{E}_{q_\phi} \left ( \log p_\theta(y) \right ) $$
$$ = \mathbb{E}_{q_\phi} \left ( \log \frac{p_\theta(y\vert z)p_\theta(z)}{p_\theta(z\vert y)} \right ) $$
$$ = \mathbb{E}_{q_\phi} \left ( \log \frac{p_\theta(y\vert z)q_\phi(z\vert y)p_\theta(z)}{p_\theta(z\vert y)q_\phi(z\vert y)} \right ) $$
$$  = KL(q_\phi(z\vert y) \Vert p_\theta(z\vert y)) - KL(q_\phi(z\vert y) \Vert p_\theta(z)) + \mathbb{E}_{q_\phi}\left(\log p_\theta(y \vert z) \right) $$


While the first KL divergence term is still intractable because it involves the posterior distribution, the other terms are. This defines a substitute objective, which is called the ELBO (Evidence Lower Bound):

$$ELBO(y)= -KL(q_\phi(z\vert y) \Vert p_\theta(z)) + \mathbb{E}_{q_\phi}\left(\log p_\theta(y \vert z) \right)$$

$$ \log p_\theta(y) = ELBO(y) + KL((q_\phi(z\vert y) \Vert p_\theta(z\vert y)) $$

Since the KL divergence term is always positive, the ELBO is a lower-bound of our true maximization objective (the log-likelihood).

## Conditional VAE
In the conditional setting, we assume the existence of an additional variable $x$ which represents the initial condition. The generative process is slightly different as the prior for $z$ is now conditioned on $x$ as well. 
* $z \sim p_\theta(z \vert x)$
* $ y \sim p_\theta(y \vert z,x)$

The objective to maximize becomes the condition log-likelihood $\log p_\theta(y \vert x)$, and in a similar fashion, we can derive a condition ELBO objective:
$$ ELBO(y\vert x) = -KL(q_\phi(z\vert y, x) \Vert p_\theta(z \vert x)) + \mathbb{E}_{q_\phi}\left(\log p_\theta(y \vert z, x) \right)$$

## Evaluation 

In order to evaluate the performance of the model, it is still useful to be able to evaluate the actual log-likelihood $\log p_\theta(y \vert x)$ and not its lower bound. In the paper, one method is to use importance sampling.

$$ \log p_\theta(y\vert x) = \log \frac {1}{S}\sum_{i=1}^{S} \frac{p_\theta(y\vert x,z^{(i)})p_\theta(z^{(i)}\vert x)}{q_\phi(z^{(i)}\vert x,y)}, z^{(i)} \sim q_\phi$$

## MNIST example

In this repository, we work with binarized MNIST examples, where pixel is either 0 or 1. The prior and posterior on the latent variables are multivariate gaussian distributions with diagonal covariance:
* $q_\phi(z \vert x,y) \sim \mathcal{N}(\mu_\phi(x,y), \sigma_\phi(x,y))$
* $p_\theta(z \vert x) \sim \mathcal{N}(\mu_\theta(x), \sigma_\theta(x))$

The marginal likelihood is a Bernoulli distribution:
* $p_\theta(y \vert x,y) \sim \mathcal{B}(q(x, y))$

### Closed-form for KL divergence 
In this particular case where the prior and posterior are multivariate gaussians with diagonal covariance, the KL divergence term can be expressed as a closed-form formula.

We give the general derivation below, where $\mu$ is the mean vector and $v$ are the diagonal covariance terms.

$$ KL(\mathcal{N}(\mu_x, v_x) \Vert \mathcal{N}(\mu_y, v_y)) = \mathbb{E}_{\mathcal{N_x}}\left( \log \frac{\mathcal{N}(\mu_x, v_x)}{\mathcal{N}(\mu_y, v_y)}\right) $$
$$ = \frac{1}{2}\mathbb{E}_{\mathcal{N_x}}\left(\sum_i \log v_{y,i} - \sum_i \log v_{x,i} + \sum_i \frac{(X_i-\mu_{y,i})^2}{v_{y,i}} - \sum_i \frac{(X_i-\mu_{x,i})^2}{v_{x,i}}\right) $$
$$ = \frac{1}{2}\sum_i (\log v_{y,i} -\log v_{x,i}) + \frac{1}{2}\mathbb{E}_{\mathcal{N_x}}\left(\sum_i \frac{(X_i-\mu_{y,i})^2}{v_{y,i}}\right) - \frac{N}{2} $$

Reparameterize gaussian variable $X$ using centered variable:
$$ = \frac{1}{2}\sum_i (\log v_{y,i} -\log v_{x,i}) + \frac{1}{2}\sum_i \mathbb{E}_{\epsilon \sim\mathcal{N}(0, 1)}\left(\frac{(\mu_{x,y} + \epsilon \sqrt{v_{x,i}}-\mu_{y,i})^2}{v_{y,i}}\right) - \frac{N}{2} $$
$$ = \frac{1}{2}(\sum_i( \log v_{y,i} -\log v_{x,i}) + \sum_i \frac{(\mu_{x,i}-\mu_{y,i})^2}{v_{y,i}} + \frac{v_{x,i}}{v_{y,i}}\mathbb{E}(\epsilon^2) + 2\mathbb{E}(\epsilon)\sqrt{v_{y,i}}(\mu_{x,i}-\mu_{y,i}) - N) $$
$$= \frac{1}{2}(\sum_i( \log v_{y,i} -\log v_{x,i}) + \sum_i \frac{(\mu_{x,i}-\mu_{y,i})^2}{v_{y,i}} + \frac{v_{x,i}}{v_{y,i}} - N) $$